In [ ]:
!pip install -q --upgrade langchain langchain-community langchain-google-genai google-generativeai faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [ ]:
import os
import json
import re
from google.colab import userdata
from google.colab import files
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Access the API key from Colab's secrets manager
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("API Key loaded successfully.")
except userdata.SecretNotFoundError:
    print("ERROR: Secret 'GOOGLE_API_KEY' not found. Please follow the instructions to add it.")

API Key loaded successfully.


In [ ]:
def create_vector_store_from_paths(file_paths):
    """Loads multiple PDFs, combines them, and creates a single vector store."""
    all_documents = []
    try:
        # Loop through each uploaded file path
        for file_path in file_paths:
            loader = PyPDFLoader(file_path)
            # Extend the master list with documents from the current PDF
            all_documents.extend(loader.load())

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        docs = text_splitter.split_documents(all_documents)

        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        vector_store = FAISS.from_documents(docs, embeddings)
        print(f"Vector store created successfully from {len(file_paths)} PDF(s).")
        return vector_store
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return None

def get_rag_chain():
    """Initializes the LLM and the prompt template for the RAG chain."""
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0, convert_system_message_to_human=True)
    prompt_template = """
    You are an expert insurance claim evaluator. Your task is to analyze a user's query against a set of relevant insurance policy clauses and determine if the claim should be approved.

    Here are the relevant policy clauses:
    ---
    {context}
    ---

    Here is the user's claim query:
    ---
    {query}
    ---

    Based *only* on the provided clauses and the user's query, perform the following steps:
    1.  Evaluate the query against the clauses.
    2.  Determine a final decision: "Approved" or "Rejected".
    3.  If approved, state the payout amount or coverage percentage if specified in the clauses. If no amount is specified, use "Not Applicable".
    4.  Provide a clear justification for your decision by referencing the specific clause(s) used.

    Return your final answer as a single, clean JSON object with no other text before or after it. The JSON object must have these exact keys: "decision", "amount", "justification".

    Final JSON Response:
    """
    rag_chain = LLMChain(
        llm=llm,
        prompt=PromptTemplate.from_template(prompt_template)
    )
    return rag_chain

def clean_and_parse_json(response_text):
    """Extracts and parses the JSON object from the AI's response."""
    # Use regex to find the JSON block, robust against markdown wrappers
    match = re.search(r'\{.*\}', response_text, re.DOTALL)
    if match:
        json_str = match.group(0)
        return json.loads(json_str)
    else:
        raise json.JSONDecodeError("No JSON object found in response", response_text, 0)

In [ ]:
# 1. Upload the files
print("Please upload all 5 of your policy PDF files.")
uploaded = files.upload()

# Check if any files were uploaded
if not uploaded:
    print("\nNo files uploaded. Please run the cell again and select your files.")
else:
    # Get the list of all uploaded filenames
    file_names = list(uploaded.keys())
    print(f"\nProcessing {len(file_names)} file(s): {', '.join(file_names)}")

    # 2. Create the combined knowledge base (vector store)
    vector_store = create_vector_store_from_paths(file_names)

    if vector_store:
        # 3. Get the AI chain
        rag_chain = get_rag_chain()

        # 4. Start the interactive query loop
        while True:
            print("\n" + "="*50)
            user_query = input("Enter your claim query (or type 'exit' to quit): ")
            if user_query.lower() == 'exit':
                break

            # Retrieve relevant documents from the combined store
            relevant_docs = vector_store.similarity_search(user_query)
            context = "\n".join([doc.page_content for doc in relevant_docs])

            # Run the RAG chain
            result = rag_chain.invoke({"context": context, "query": user_query})

            # Display the result
            print("\n--- Evaluation Result ---")
            try:
                response_json = clean_and_parse_json(result.get('text', '{}'))
                print(json.dumps(response_json, indent=2))
            except json.JSONDecodeError:
                print("Failed to get a valid JSON response from the AI.")
                print("\n--- Raw Output ---")
                print(result.get('text', 'No output.'))

print("\nSession ended.")

Please upload all 5 of your policy PDF files.


Saving rifa resume.pdf to rifa resume.pdf

Processing 1 file(s): rifa resume.pdf


/tmp/ipython-input-1622391767.py:48: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  rag_chain = LLMChain(


Vector store created successfully from 1 PDF(s).

Enter your claim query (or type 'exit' to quit): WHAT IS RIFA 12TH PASSING YEAR


/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



--- Evaluation Result ---
{
  "decision": "Approved",
  "amount": "Not Applicable",
  "justification": "The provided text states that RIFA completed their 12th standard in 2022."
}

Enter your claim query (or type 'exit' to quit): exit

Session ended.
